In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Aşırı öğrenmeyi(overfitting) azaltmanın bit yolu modeli düzenlileştirmektir(yani modeli kısıtlamaktır.). Örneğin bir polinom modeli kısıtlamanın basit bir yolu polinom derecelerinin sayısını azaltmaktır.

Doğrusal bir model için düzenlileştirme,genellikle modelin katsayılarını(ağırlıklarını - weights) kısıtlayarak gerçekleştirilir. Modelin katsayılarını kısıtlamak için 3 farklı yol uygulayan Ridge Regression, Lasso Regression, ve Elastic Net'e bakacağız:

# Ridge Regresyon

Ridge Regresyon(Tikhonov regularization da denir), Doğrusal Regresyonun düzenlileştirilmiş bir versiyonudur: $\alpha \sum_{i=1}^{n} \theta_{i}^{2}$ denklemine eşit olan düzenlileştirme terimi, maliyet fonksiyonuna eklenir. Bu,öğrenme algoritmasını hem verilere uymaya ,hemde modelin ağırlıklarını mümkün olduğunca küçük tutmaya zorlar. Dikkat edilmesi gereken bir nokta: düzenlileştirme terimi,sadece eğitim sırasında maliyet fonksiyonuna eklenmelidir. Modeli eğittikten sonra,modeli değerlendirmek için düzenleştirilmemiş performans ölçütleri kullanabilirsiniz.

$\alpha$ hiperparametresi,modeli ne kadar düzenlileştirmek istediğinizi kontrol eder. Eğer $\alpha=0$ ise, Ridge Regresyon bir Doğrusal Regresyon olur. Eğer $\theta$ çok büyük ise tüm ağırlıklar sıfıra çok yakın olacaktır.
Ridge Regresyon maliyet fonksiyonu:

$$RSS_{RIDGE} = \sum_{i=1}^{m}(h_{\theta}(x_{i})-y_{i})^{2} + \alpha \sum_{j=1}^{n}\theta^{2}_{j}$$

Bias terimi $\theta_{0}$'ın düzenlileştirilmediğine dikkat edin(toplama i=1'den başlıyor.).

---------------

# Uyarı

Girdi özelliklerinin ölçeğine duyarlı olduğu için Ridge Regresyon uygulamadan önce verileri ölçeklendirmek (örneğin,StandartScaller kullanarak) önemlidir. Bu,çoğu doğrusal model için geçerlidir.

---------------

Doğrusal Regresyon'da olduğu gibi,Ridge Regresyonu uygulamak için kapalı-form bir denklem(doğrusal regresyondaki Normal Denklem gibi) veya Gradient Descent kullanabiliriz.

Ridge Regresyon kapalı-form çözümü:

$$\hat{\theta} = (X^{T}X+\alpha A)^{-1}X^{T}y$$

NOT: Denklemdeki A:(n+1)x(n+1)'lik birim matristir.

In [ ]:
# Scikit-Learn kullanarak Ridge Regresyon uygulaması:

# Önce rasggele veriler oluşturalım:
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt

np.random.seed(42)

m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)

# Ridge Regresyon (kapalı-form çözüm kullanarak)
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1, solver="cholesky")
ridge_reg.fit(X, y)
ridge_reg.predict([[1.5]])

In [ ]:
# Ridge Regresyon (Stochastic Gradient Descent kullanarak):

from sklearn.linear_model import SGDRegressor
sgd_reg_ridge = SGDRegressor(penalty="l2")
sgd_reg_ridge.fit(X, y.ravel())
sgd_reg_ridge.predict([[1.5]])

penalty hiperparametresi kullanılacak düzenlileştirme teriminin tipini belirler. "l2" olarak belirtirsek,maliyet fonksiyonuna model ağırlıklarının karesi eklenir.Buna "l2" norm da denir ve maliyet fonksiyonuna model ağırlıklarını l2 normunun eklenmesi basitçe Ridge Regresyondur.

# Lasso Regresyon

Least Absolute Shrinkage and Selection Operator Regression(kısaca Lasso Regression), Doğrusal Regresyonun düzenlileştirilmiş başka bir versiyonudur: Ridge Regresyon'da olduğu gibi, maliyet fonksiyonuna bir düzenlileştirme terimi eklenir. Ancak Lasso Regresyon ağırlık vektörünün "l1" normunu kullanır.

Lasso Regresyon Maliyet Fonksiyonu:

$$RSS_{LASSO} = \sum_{i=1}^{m}(h_{\theta}(x_{i})-y_{i})^{2} + \alpha \sum_{j=1}^{n}|\theta_{j}|$$

Lasso Regresyonun önemli bir özelliği,en az öneme sahip özellikleri sıfıra eşitleyerek onları eleme eğiliminde olmasıdır.Başka bir deyişle, Lasso Regresyon otomatik olarak özellik seçimi gerçekleştirir.

In [ ]:
# Lasso sınıfını kullanan küçük bir Scikit-Learn örneği:

from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X,y)
lasso_reg.predict([[1.5]])

In [ ]:
# Lasso Regresyon (Stochastic Gradient Descent kullanarak):

from sklearn.linear_model import SGDRegressor
sgd_reg_lasso = SGDRegressor(penalty="l1")
sgd_reg_lasso.fit(X, y.ravel())
sgd_reg_lasso.predict([[1.5]])

# Elastic Net

Elastic Net, Ridge Regresyon ile Lasso Regresyonun ortasındadır. Elastic Net'in maliyet fonksiyonu Ridge Regresyonun maliyet fonksiyonu ile Lasso Regresyonun maliyet fonksiyonunun karışımıdır ve bu karışımı,r hiperparametresini kullanarak kontrol edebilirsiniz. r=0 olduğunda Elastic Net, Ridge Regresyona denktir ve r=1 olduğunda Elastic Net, Lasso Regresyona denktir.

Elastic Net Maliyet Fonksiyonu:

$$J(\theta)=MSE(\theta)+r \alpha \sum_{i=1}^n |\theta_{i}|+\frac{1-r}{2} \alpha \sum_{i=1}^{n} \theta_{i}^{2}$$

In [ ]:
# Scikit-Learn’ün Elastik Net sınıfını kullanan kısa bir örnek (l1_ratio, r karışım oranına karşılık gelir):

from sklearn.linear_model import ElasticNet
elastic_net=ElasticNet(l1_ratio=0.5,alpha=0.1)
elastic_net.fit(X,y)
elastic_net.predict([[1.5]])

Peki ne zaman düz Doğrusal Regresyon(yani herhangi bir düzenlileştirme olmadan),Ridge,Lasso veya Elastic Net kullanmalısınız? En azından biraz düzenlileştirme olması neredeyse her zaman tercih edilir bu sebeple düz Doğrusal Regresyondan kaçınmalısınız. Ridge iyi bir varsayılandır ancak eğer sadece birkaç özelliğin faydalı,işe yarar olduğunu düşünüyorsanız Lasso veya Elastic Net tercih etmelisiniz çünkü Lasso ve Elastic Net faydasız,işe yaramaz özelliklerin ağırlıklarını sıfır yapma eğilimindedirler. Genel olarak Lasso'ya karşı Elastic Net tercih edilir çünkü Lasso, özellik sayısı eğitim örneği sayısından daha fazla olduğunda veya birkaç özellik güçlü bir şekilde ilişkili(correlated) olduğunda düzensiz davranabilir.